# Performance Test Movenet
## Loading model

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import psutil
import time

# Carregar o modelo MoveNet
module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192

# Função para inferência do MoveNet
def movenet(input_image):
    model = module.signatures['serving_default']
    input_image = tf.cast(input_image, dtype=tf.int32)
    outputs = model(input_image)
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

# Função para processar um frame
def process_frame(frame):
    input_image = tf.expand_dims(frame, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
    keypoints_with_scores = movenet(input_image)
    return keypoints_with_scores

# Função para monitorar uso de CPU e memória
def monitor_performance():
    cpu_usage = psutil.cpu_percent(interval=0.1)
    memory_usage = psutil.virtual_memory().used / (1024 * 1024)  # Convertendo para MB
    return cpu_usage, memory_usage



2024-12-06 07:00:57.032169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 07:01:00.453308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 07:01:01.059100: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 07:01:06.632451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-06 07:01:18.816459: W tensorflow/compiler/tf2

Setting up video

In [2]:
# Abrir o vídeo
video_path = "pushupseries.mp4"  # Substitua pelo caminho do seu vídeo
cap = cv2.VideoCapture(video_path)

# Variáveis de desempenho
cpu_usage_list = []
memory_usage_list = []
frame_count = 0

## Proessing video

In [3]:
# Processar o vídeo
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Redimensionar o frame para o tamanho esperado pelo modelo
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = tf.convert_to_tensor(frame_rgb, dtype=tf.float32)

    # Medir tempo de inferência
    start_time = time.time()
    keypoints_with_scores = process_frame(frame_tensor)
    end_time = time.time()

    # Monitorar uso de CPU e memória
    cpu_usage, memory_usage = monitor_performance()
    cpu_usage_list.append(cpu_usage)
    memory_usage_list.append(memory_usage)

    # Exibir resultados no terminal
    print(f"Frame {frame_count}:")
    print(f"Inference time: {end_time - start_time:.2f}s")
    print(f"CPU usage: {cpu_usage:.2f}%")
    print(f"Memory usage: {memory_usage:.2f} MB")

cap.release()

Frame 1:
Inference time: 1.34s
CPU usage: 1.70%
Memory usage: 6269.51 MB
Frame 2:
Inference time: 0.01s
CPU usage: 7.40%
Memory usage: 6280.68 MB
Frame 3:
Inference time: 0.01s
CPU usage: 4.90%
Memory usage: 6288.55 MB
Frame 4:
Inference time: 0.01s
CPU usage: 3.40%
Memory usage: 6296.43 MB
Frame 5:
Inference time: 0.01s
CPU usage: 3.40%
Memory usage: 6304.79 MB
Frame 6:
Inference time: 0.01s
CPU usage: 2.50%
Memory usage: 6307.74 MB
Frame 7:
Inference time: 0.01s
CPU usage: 5.00%
Memory usage: 6310.67 MB
Frame 8:
Inference time: 0.01s
CPU usage: 2.50%
Memory usage: 6317.07 MB
Frame 9:
Inference time: 0.01s
CPU usage: 5.90%
Memory usage: 6324.15 MB
Frame 10:
Inference time: 0.01s
CPU usage: 2.50%
Memory usage: 6325.62 MB
Frame 11:
Inference time: 0.01s
CPU usage: 1.70%
Memory usage: 6325.87 MB
Frame 12:
Inference time: 0.01s
CPU usage: 2.50%
Memory usage: 6325.87 MB
Frame 13:
Inference time: 0.01s
CPU usage: 2.50%
Memory usage: 6326.11 MB
Frame 14:
Inference time: 0.01s
CPU usage: 5.10

Results

In [4]:
# Calcular métricas finais
avg_cpu_usage = np.mean(cpu_usage_list)
avg_memory_usage = np.mean(memory_usage_list)

print("\nPerformance Summary:")
print(f"Average CPU usage: {avg_cpu_usage:.2f}%")
print(f"Average Memory usage: {avg_memory_usage:.2f} MB")



Performance Summary:
Average CPU usage: 3.73%
Average Memory usage: 6523.57 MB
